In [1]:
# Obtiene las variables climáticas, socioeconómicas y de gobernanza
# y cálcula índices para cada categoría

import numpy as np
import pandas as pd

from scipy import stats

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Datos

# Carpetas
path_catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"

# Código de país
ix = "alpha-3"

# Cargamos archivos y creamos copias de trabajo
df_c = pd.read_csv(path_catalog)
df_iso = pd.read_csv(iso).set_index(ix)
data_0 = pd.DataFrame(index = df_iso.index)
data_s   = data_0.copy()
data_c   = data_0.copy()
data_g   = data_0.copy()
data_q_s = data_0.copy()
data_q_c = data_0.copy()
data_q_g = data_0.copy()
df_v = pd.read_csv(path_v)

In [3]:
# Carga la variable y calcula el índice

deciles_n = [ f"../results/Deciles_{x}.csv" 
    for x in df_v["Category"].unique() ]
values_n  = [ f"../results/Values_{x}.csv" 
    for x in df_v["Category"].unique() ]

# Iteramos para socioeconómico, climático, y de gobernanza
for cat in df_v["Category"].unique():
    data   = data_0.copy()
    data_q = data_0.copy()

    # Para cada variable de la categoría
    for row in df_v[ (df_v["Category"] == cat) ].itertuples():

        v = row.Variable
        # Para datos del Banco mundial hay que quitar filas al csv
        if row.Origin == "World Bank": nr = 3
        else: nr = 0
        
        # Cargamos el archivo
        df = pd.read_csv( "../../"
            + df_c.loc[df_c["ID"]==row.ID, "Path"].iloc[0]
            + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
            skiprows = nr, low_memory = False )
        
        # Procesamiento de datos de la OCDE
        if row.Origin == "OECD":
            # Se escoge la variable
            df = df[ df["MEASURE"] == v ]
            df = df.rename(columns = {"REF_AREA": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            df_i = df[ (df["MEASURE"]==v) ]
            
            # Se escogen los datos específicos para cada variable
            if v == "POP_NETMIGR":
                df_i["OBS_VALUE"] = np.abs( df_i["OBS_VALUE"] )
            elif v in ["HD_POP_EXP", "HD_TN_POP_EXP", "ID_POP_EXP",
                "99PT_LAND_EXP", "99PT_CROP_EXP", "EP_LAND_EXP"]:
                df_i = df_i[ (df_i["DURATION"]==row.DURATION) ]
            elif v in ["TEMP_CHANGE", "ED_CROP_ANOM", "EP_CHANGE"]:
                df_i = df_i.set_index(ix)
                df_i["OBS_VALUE"] = df_i.reset_index()[
                    [ix, "OBS_VALUE"] ].groupby(ix).mean()
                df_i = df_i.reset_index()
            elif v == "UTCI_POP_EXP":
                df_i = df_i[ (df_i["HEAT_STRESS"]==row.HEAT_STRESS) ]
            elif v in ["CF_BUILT_EXP", "CF_POP_EXP", "CF_CROP_EXP",
                "RF_BUILT_EXP", "RF_POP_EXP", "RF_CROP_EXP"]:
                df_i = df_i[ (df_i["RET_PERIOD"]==row.RET_PERIOD) ]
            elif v == "BURN_LAND_AREA":
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            elif v in ["CY_BUILT_EXP", "CY_POP_EXP"]:
                df_i = df_i[
                    (df["HURRICANE_WIND_SCALE"]==row.HURRICANE_WIND_SCALE) ]
            elif v == "POP_EXP_POL":
                df_i = df_i[ (df_i["EXPOSURE_LEVEL"]==row.EXPOSURE_LEVEL) ]
            elif v == "ECRATE":
                df_i = df_i[ (df_i["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            elif v in ["GPAT_DE", "WATER_FW"]:
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            
            # Escogemos solo el dato más reciente
            ''' Analizar el calcular una climatología '''
            data[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]

        # Procesamiento de datos de Global Data Lab
        elif row.Origin == "Global Data Lab":
            df = df.rename(columns = {"ISO_Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
        
        # Procesamiento de datos del Banco Mundial
        elif row.Origin == "World Bank":
            df = df.rename(columns = {"Country Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df.set_index(ix).iloc[ :, 4:-1
                ].ffill(axis = 1).iloc[:, -1]
        
        # Procesamiento de datos de la UNESCO
        elif row.Origin == "UNESCO UIS":
            if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
                df.columns = [ x.lower() for x in df.columns ]
            df = df.rename(columns = {"country_id": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            if row.PERCENTAGE == "Yes": pop = data["200101"]
            else: pop = 1
            data[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
                ).drop_duplicates(ix, keep="last"
                ).set_index(ix)["value"] / pop )
        
        # El índice se calcula como el percentile of score
        ''' Revisar como va a funcionar para proyecciones futuras '''
        data_q[v] = stats.percentileofscore(
            data[v], data[v], nan_policy = "omit" ) / 10
        # Se ajusta el índice para que más siempre sea mejor
        if row.Negative_indicator:
            data_q[v] = 10 - data_q[v]

    # Calculamos el promedio y la cantidad de datos para cada categoría
    data_q["Mean"] = data_q.mean(axis = 1)
    data_q["Count"] = data_q.count(axis = 1) - 1
    # Escogemos solo los países que sí tengan datos
    data_q = data_q[ data_q["Count"] > 0 ].sort_values("Mean")

    # Asignamos los valores a la tabla de cada categoría
    if   cat == df_v["Category"].unique()[0]:
        data_q_s = data_q.copy()
        data_s   = data.copy()
        data_q_s.to_csv(deciles_n[0], index = True)
        data_s.to_csv(values_n[0], index = True)
    elif cat == df_v["Category"].unique()[1]:
        data_q_c = data_q.copy()
        data_c   = data.copy()
        data_q_c.to_csv(deciles_n[1], index = True)
        data_c.to_csv(values_n[1], index = True)
    elif cat == df_v["Category"].unique()[2]:
        data_q_g = data_q.copy()
        data_g   = data.copy()
        data_q_g.to_csv(deciles_n[2], index = True)
        data_g.to_csv(values_n[2], index = True)

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,POP_NETMIGR,Subnational HDI,Subnational GDI,Gini index,Mean,Count
alpha-3,,,,,,,,,,,,,,,,,,,,,
VAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.048780,NaN,NaN,NaN,NaN,0.048780,1
SSD,3.605150,9.656652,8.841202,NaN,1.179245,1.276596,0.361446,9.761905,0.758294,2.976744,...,0.000000,0.00,0.000000,0.341463,2.046512,0.107527,0.535714,1.497006,2.256625,22
AGO,1.716738,7.725322,9.227468,0.583333,2.688679,1.702128,1.987952,8.714286,0.284360,0.279070,...,1.732283,1.84,0.967742,1.365854,2.186047,2.419355,2.172619,0.479042,2.334673,24
TCD,2.832618,8.412017,9.785408,8.000000,0.660377,2.411348,1.084337,9.952381,0.094787,0.232558,...,0.157480,0.08,0.161290,1.658537,4.674419,0.161290,0.178571,4.161677,2.396878,25
PNG,3.948498,8.025751,8.326180,0.750000,2.783019,2.836879,0.963855,7.714286,2.511848,2.232558,...,0.944882,1.04,1.532258,0.536585,4.093023,1.962366,2.440476,2.215569,2.524022,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SXM,9.055794,9.785408,3.905579,NaN,8.301887,NaN,NaN,3.904762,7.345972,3.627907,...,NaN,NaN,NaN,NaN,8.813953,NaN,NaN,NaN,7.164917,9
PLW,9.442060,4.120172,4.420601,NaN,6.037736,NaN,NaN,6.857143,9.857820,7.627907,...,NaN,NaN,NaN,9.463415,9.813953,NaN,NaN,NaN,7.192006,12
ISL,7.596567,3.175966,2.703863,NaN,9.433962,8.758865,9.728916,0.571429,5.687204,1.069767,...,NaN,NaN,NaN,9.317073,8.000000,9.892473,6.011905,9.520958,7.242548,19


In [4]:
# Agregamos valores al resumen de países

# Nombres de índices
res_names = [ "Socioeconomic resilience",
    "Climate resilience", "Governance resilience" ]

# Creamos columnas con los índices, eliminamos los países con pocos datos.
df_iso[ res_names[0]  ] = data_q_s.loc[
    data_q_s["Count"] > 10, "Mean" ]
df_iso[ res_names[1]  ] = data_q_c.loc[
    data_q_c["Count"] > 10, "Mean" ]
# Hay que ser muy laxos en gobernanza para 
# incluir a Bután que tiene objetivo Net Zero
df_iso[ res_names[2]  ] = data_q_g.loc[
    data_q_g["Count"] > 2, "Mean" ]

# Cargamos evaluación Net Zero
df_nz = pd.read_csv( "../../" +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Path" ].iloc[1] +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Filename" ].iloc[1],
    usecols = ["ISO", "Overall Rating"] )
df_nz = df_nz.rename( columns = {"ISO": ix} ).set_index(ix)
df_iso["Net Zero Rating"] = df_nz["Overall Rating"]
df_iso["Net Zero Rating"] = df_iso["Net Zero Rating"].where(
    ~df_iso["Net Zero Rating"].isnull(), "No target" )

# Limpiamos la tabla y guardamos
df_res = df_iso.drop( ["country-code", "iso_3166-2", "intermediate-region",
    "region-code", "sub-region-code", "intermediate-region-code"], axis = 1 )
# Nos ponemos más estrictos con el valor de gobernanza
df_res2 = df_res.copy()
df_res2[ res_names[2]  ] = data_q_g.loc[
    data_q_g["Count"] > 5, "Mean" ]
df_res2.to_csv( "../results/countries_values.csv", index = True )

# Escogemos solo los países con objetivo Net Zero
df_rnz = df_res.loc[ df_res["Net Zero Rating"] != "No target",
    ["name", "Net Zero Rating", "alpha-2"] + res_names ]
df_rnz.to_csv( "../results/countries_values_net_zero.csv", index = True )

df_rnz

,name,Net Zero Rating,alpha-2,Socioeconomic resilience,Climate resilience,Governance resilience
alpha-3,,,,,,
ARG,Argentina,Poor,AR,5.355912,3.699203,5.357407
AUS,Australia,Poor,AU,6.173538,4.323282,5.009759
BTN,Bhutan,Information incomplete,BT,5.707487,4.988717,3.606019
BRA,Brazil,Information incomplete,BR,5.530113,4.236506,5.641233
CAN,Canada,Average,CA,5.882945,4.285114,4.982974
CHL,Chile,Acceptable,CL,5.672913,4.506615,5.887933
CHN,China,Poor,CN,4.689794,3.093866,4.745535
COL,Colombia,Acceptable,CO,4.391096,4.593764,5.883822
CRI,Costa Rica,Acceptable,CR,6.263940,5.403217,5.182518


In [5]:
# Graficamos los indicadores para los países con objetivo Net Zero

# Reodenamos las primeras filas para tener la simbología correcta
df_g = df_rnz.copy()
n = list(range(df_rnz.shape[0]))
n.remove(0)
n.remove(2)
n.remove(4)
n.remove(5)
df_g = df_g.iloc[[2, 0, 4, 5] + n ]
for i in res_names:
    df_g[i] = df_rnz[i].apply(lambda x: float(f"{x:.2f}"))

# Graficamos las coordenadas de los países y el Net Zero Rating
fig = px.scatter( df_g, x = res_names[0], y = res_names[1],
    hover_name = "name", hover_data = res_names + ["Net Zero Rating"],
    size = res_names[2], color = "Net Zero Rating", opacity = 1, size_max = 25,
    color_discrete_sequence=["#808080", "#8B0000", "#DAA520", "#006400"]
)
# Hacemos que el scatter sea invisible
#fig.update_traces( marker_color = "rgba(0,0,0,0)" )

'''Agregar Net Zero Rating como círculo de color'''
# Agregamos la bandera de cada país
for i, row in df_rnz.iterrows():
    country_iso = row["alpha-2"]
    fig.add_layout_image(
        {
            "source": "https://raw.githubusercontent.com/matahombres/"
            + f"CSS-Country-Flags-Rounded/master/flags/{country_iso}.png",
            "xref": "x", "yref": "y", "xanchor": "center", "yanchor": "middle",
            "x": row[res_names[0]], "y": row[res_names[1]],
            "sizex": row[res_names[2]]/25, "sizey": row[res_names[2]]/25,
            "sizing": "contain",
            #"bordercolor":  "green",
            "opacity": 1,
            "layer": "above"
        }
    )

# Ajustes de visualización
fig.update_layout( height = 600, width = 1000, plot_bgcolor = "#dfdfdf" )

fig.show()